#Bilgan Kiris
#Week 6 - File Ingestion & Schema Validation
#November 8th, 2024

#Setting up environment and importing libraries

In [ ]:
# Install required libraries
!pip install dask modin[ray] pyyaml ray

# installing required libraries
import pandas as pd
import numpy as np
import os
import dask.dataframe as dd
import modin.pandas as mpd
import ray
import time
import yaml

# initialize ray for modin
ray.init()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.5 MB/s eta 0:00:00


2024-11-08 23:57:43,624	INFO worker.py:1816 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.38.0


In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  print(f"uploaded{filename}")

file_path = list(uploaded.keys())[0]

# for this assignment I have found a 3GB dataset from Kaggle called "Fashion Dataset"
# URL: https://www.kaggle.com/datasets/hiteshsuthar101/myntra-fashion-product-dataset

Saving Fashion Dataset.csv to Fashion Dataset.csv
uploadedFashion Dataset.csv


In [ ]:
import pandas as pd

# loading the uploaded csv file
data = pd.read_csv(file_path)

# displaying basic information and verifying
print("Data loaded successfully")
data.info()

Data loaded successfully
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14330 entries, 0 to 14329
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    14330 non-null  int64  
 1   p_id          14312 non-null  float64
 2   name          14312 non-null  object 
 3   price         14312 non-null  float64
 4   colour        14309 non-null  object 
 5   brand         14312 non-null  object 
 6   img           14312 non-null  object 
 7   ratingCount   6581 non-null   float64
 8   avg_rating    6581 non-null   float64
 9   description   14312 non-null  object 
 10  p_attributes  14312 non-null  object 
dtypes: float64(4), int64(1), object(6)
memory usage: 1.2+ MB


#Expanding the Dataset

In [ ]:
# checking the current file size
file_size = os.path.getsize(file_path) / (1024 * 1024)
print(f"Current file size: {file_size:.2f} MB")

Current file size: 15.57 MB


#Different Reading Methods (Pandas, Dask, Mobin)

In [ ]:
# Pandas
start_time = time.time()
pandas_df = pd.read_csv(file_path)
end_time = time.time()
pandas_time = end_time - start_time
print(f"Pandas time: {pandas_time:.2f} seconds")

Pandas time: 0.27 seconds


In [ ]:
# Dask
start_time = time.time()
dask_df = dd.read_csv(file_path)
end_time = time.time()
dask_time = end_time - start_time
print(f"Dask time: {dask_time:.2f} seconds")

Dask time: 0.02 seconds


In [ ]:
# Modin
start_time = time.time()
modin_df = mpd.read_csv(file_path)
end_time = time.time()
modin_time = end_time - start_time
print(f"Modin time: {modin_time:.2f} seconds")

Modin time: 1.93 seconds


Comparing Results
- Dask having the fastest reading time with 0.02 seconds and provides the best computational effiency.

#Cleaning and Validating Data

In [19]:
import dask.dataframe as dd

# re-read
df_dask = dd.read_csv(file_path)

# cleaning column name by replacing any special characters and spaces.
df_dask.columns = df_dask.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)
df_dask.columns = df_dask.columns.str.replace(' ', '_')

# display
df_dask.head()

,Unnamed__0,p_id,name,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,0,17048614.0,Khushal K Women Black Ethnic Motifs Printed Ku...,5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,1,16524740.0,InWeave Women Orange Solid Kurta with Palazzos...,5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,2,16331376.0,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,3,14709966.0,Nayo Women Red Floral Printed Kurta With Trous...,3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,4,11056154.0,AHIKA Women Black & Green Printed Straight Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


#Creating YAML File for Schema

#Checking Missing Value

In [20]:
df_dask.isna().sum()

Dask Series Structure:
npartitions=1
Unnamed__0     int64
ratingCount      ...
dtype: int64
Dask Name: to_pyarrow_string, 8 graph layers

#Generating the Summary of the File

In [36]:
import gzip

# Define the schema details
schema = {
    "columns": ['Unnamed: 0', 'p_id', 'name', 'price', 'colour', 'brand', 'img',
        'ratingCount', 'avg_rating', 'description', 'p_attributes'],
    "separator": ","
}

# Write schema to YAML file
with open("schema.yml", "w") as f:
    yaml.dump(schema, f)

# Load the dataset using Dask
df_dask = dd.read_csv(file_path)

# Load and parse the YAML configuration file
def load_yaml_config(yaml_file):
    with open(yaml_file, "r") as file:
        return yaml.safe_load(file)

config = load_yaml_config("schema.yml")

# Add "p_attributes" column if it doesn't exist in the DataFrame
if "p_attributes" not in df_dask.columns:
    df_dask["p_attributes"] = ""

# Validate DataFrame structure against YAML configuration
def validate_columns(dataframe, config_columns):
    if list(dataframe.columns) == config_columns:
        print("Column validation succeeded.")
    else:
        print("Column validation failed: Column names or order mismatch.")

validate_columns(df_dask, config["columns"])

# Select a subset and save as pipe-separated file
df_sample = df_dask.head(2000)

# Convert the subset to a Dask DataFrame with one partition
df_sample_dask = dd.from_pandas(df_sample, npartitions=1)

# Save to gzipped file
output_filename = "FashionDatasetSamplePipe.gz"
df_sample_dask.to_csv(output_filename, sep="|", index=False, compression="gzip", single_file=True)

# Generate and print the file summary
def generate_file_summary(dataframe, filename):
    summary = {
        "Total number of rows": dataframe.shape[0],
        "Total number of columns": dataframe.shape[1],
        "File size (MB)": os.path.getsize(filename) / (1024 * 1024)
    }
    return summary

summary = generate_file_summary(df_sample, output_filename)
print("File Summary:", summary)

Column validation succeeded.
File Summary: {'Total number of rows': 2000, 'Total number of columns': 11, 'File size (MB)': 0.41729068756103516}
